In [1]:
import logging

def use_logging(func):
    
    def wrapper():
        logging.warning("%s is running" % func.__name__)
        return func()
    return wrapper

@use_logging
def foo():
    print("i am foo")

foo()

i am foo


In [2]:
def use_logging(level):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if level == "warn":
                logging.warning("%s is running" % func.__name__)
            elif level == "info":
                logging.info("%s is running" % func.__name__)
            return func(*args)
        return wrapper

    return decorator

@use_logging(level="warn")
def foo(name='foo'):
    print("i am %s" % name)

foo()

i am foo


In [1]:
import numpy as np
import torch
import gym
import argparse
import os


from utils import utils
from agents import TD3
from envs.env import LinearFitting
from envs.linear_fitting import Linear_model
from agents import DDPG






In /Users/wangtaiyi/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/wangtaiyi/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/wangtaiyi/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/wangtaiyi/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/wangtaiyi/opt/anac

In [36]:
dict_a = {"k":[1,2],"b":[2,3]}
print(list(dict_a.values()))
for key in dict_a.keys():
    print(dict_a[key])


[[1, 2], [2, 3]]
[1, 2]
[2, 3]


In [43]:
print(list(dict_a.values()))
a = list(dict_a.values())


[[1, 2], [2, 3]]


In [48]:
from copy import deepcopy
import itertools
import numpy as np

# print(itertools.product(list(dict_a.values())))
r = []
b= deepcopy(a)
result = b[0]
for i in range(len(a)):
    if len(b) != 1:
       b.pop(0)
       result = itertools.product(result,b[0])
    else:
        break
    

for [k,b] in result:

    print([k,b])

[1, 2]
[1, 3]
[2, 2]
[2, 3]


In [45]:
print(a)

[[1, 2], [2, 3]]


In [62]:
from sklearn import datasets
from sklearn.preprocessing import normalize, scale
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from hyperopt import fmin,tpe,hp,partial,Trials
import matplotlib.pyplot as plt


iris = datasets.load_iris()
X = np.array(range(150))
y = iris.target

def hyperopt_model_score_svm(params):
    X_ = X[:]

    if 'normalize' in params:
        if params['normalize'] == 1:
            X_ = normalize(X_)
        del params['normalize']

    if 'scale' in params:
        if params['scale'] == 1:
            X_ = scale(X_)
        del params['scale']

    print(params['gamma'])
    print(y.shape)
    print(X_.shape)

    return y-X_*params['gamma']+params['C']


space_svm = {
    'C': hp.uniform('C', 0, 20),
    'kernel': hp.choice('kernel', ['linear', 'sigmoid', 'poly', 'rbf']),
    'gamma': hp.uniform('gamma', 0, 20),
    'scale': hp.choice('scale', [0, 1]),
    'normalize': hp.choice('normalize', [0, 1])
}


def f_svm(params):
    acc = hyperopt_model_score_svm(params)
    return -acc


trials = Trials()
best = fmin(f_svm, space_svm, algo=tpe.suggest, max_evals=1000, trials=trials)
print('best:')
print(best)

18.116870746811113                                      
  0%|          | 0/1000 [00:00<?, ?trial/s, best loss=?]

job exception: name 'X_' is not defined



  0%|          | 0/1000 [00:00<?, ?trial/s, best loss=?]


NameError: name 'X_' is not defined

In [67]:
import subprocess

output = subprocess.getstatusoutput('ls')

output

0

In [12]:
import math
def gauss_kernal(sample_scale,sigma):
    tempa = sample_scale.reshape(-1, 1) 
    tempb = np.transpose(tempa)
    temp = tempa - tempb
    gauss = math.exp(- temp**2 / (2 * sigma**2))
    return gauss

In [17]:
import numpy as np
sample_locate = []
hyperparameters ={}

param_grid = {
        'k': list(np.arange(0,10,1e-3)),
        'b': list(np.arange(0,10,1e-3))
        }

sigma = 0.2
for key in param_grid.keys():
        sample_locate.append(np.linspace(param_grid[key][0],param_grid[key][-1], 1))
        v = np.random.normal(np.zeros(1),gauss_kernal(sample_locate[-1], sigma),1) 
        hyperparameters[key] = v

print(hyperparameters)

{'k': array([0.7518452]), 'b': array([-2.19427538])}


In [12]:
import copy

In [14]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == "cuda":
    torch.cuda.set_device(0)

# Implementation of Deep Deterministic Policy Gradients (DDPG)
# Paper: https://arxiv.org/abs/1509.02971
# [Not the implementation used in the TD3 paper]


class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()

        self.l1 = nn.Linear(state_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, action_dim)

        self.max_action = max_action


    def forward(self, state):
        a = F.relu(self.l1(state))
        a = F.relu(self.l2(a))
        return self.max_action * torch.tanh(self.l3(a))


class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()

        self.l1 = nn.Linear(state_dim, 400)
        self.l2 = nn.Linear(400 + action_dim, 300)
        self.l3 = nn.Linear(300, 1)


    def forward(self, state, action):
        q = F.relu(self.l1(state))
        q = F.relu(self.l2(torch.cat([q, action], 1)))
        return self.l3(q)


class DDPG(object):
    

    def __init__(self, state_dim, action_dim, max_action, discount=0.99, tau=0.001):
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = copy.deepcopy(self.actor)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=1e-4)

        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = copy.deepcopy(self.critic)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), weight_decay=1e-2)

        self.discount = discount
        self.tau = tau


    def select_action(self, state):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()


    def train(self, replay_buffer, batch_size=64):
        # Sample replay buffer 
        state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)

        # Compute the target Q value
        target_Q = self.critic_target(next_state, self.actor_target(next_state))
        target_Q = reward + (not_done * self.discount * target_Q).detach()

        # Get current Q estimate
        current_Q = self.critic(state, action)

        # Compute critic loss
        critic_loss = F.mse_loss(current_Q, target_Q)

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Compute actor loss
        actor_loss = -self.critic(state, self.actor(state)).mean()

        # Optimize the actor 
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update the frozen target models
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


    def save(self, filename):
        torch.save(self.critic.state_dict(), filename + "_critic")
        torch.save(self.critic_optimizer.state_dict(), filename + "_critic_optimizer")

        torch.save(self.actor.state_dict(), filename + "_actor")
        torch.save(self.actor_optimizer.state_dict(), filename + "_actor_optimizer")


    def load(self, filename):
        self.critic.load_state_dict(torch.load(filename + "_critic"))
        self.critic_optimizer.load_state_dict(torch.load(filename + "_critic_optimizer"))
        self.critic_target = copy.deepcopy(self.critic)

        self.actor.load_state_dict(torch.load(filename + "_actor"))
        self.actor_optimizer.load_state_dict(torch.load(filename + "_actor_optimizer"))
        self.actor_target = copy.deepcopy(self.actor)


kwargs = {
        "state_dim": 2,
        "action_dim": 2,
        "max_action": 3,
        "discount": 1,
        "tau": 3,
    }

    


policy = DDPG(**kwargs)
        
    

In [8]:
class Foo(object):
    def __init__(self, func):
        self._func = func

    def __call__(self):
        print ('class decorator runing')
        self._func()
        print ('class decorator ending')

@Foo
def bar():
    print ('bar')

bar()

class decorator runing
bar
class decorator ending


In [18]:
# 装饰器
def logged(func):
    def with_logging(*args, **kwargs):
        print(func.__name__ )     # 输出 'with_logging'
        print(func.__doc__)      # 输出 None
        return func(*args, **kwargs)
    return with_logging

# 函数
@logged
def f(x):
   """does some math"""
   return x + x * x

logged(f)

<function __main__.logged.<locals>.with_logging(*args, **kwargs)>

In [16]:
from functools import wraps
def logged(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__)      # 输出 'f'
        print(func.__doc__)       # 输出 'does some math'
        return func(*args, **kwargs)
    return with_logging

@logged
def f(x):
   """does some math"""
   return x + x * x

f(2)

f
does some math


6

In [77]:
import os
# os.system('g++ -o ./Index/PGM/test.so -shared -fPIC ./Index/PGM/index_test.cpp')
os.system('g++ ./Index/PGM/index_test.cpp  -std=c++17 -o ./Index/PGM/exe_pgm_index')
os.system('./Index/PGM/exe_pgm_index ./Index/PGM/data_0.txt')

In file included from ./Index/PGM/index_test.cpp:17:
In file included from ./Index/PGM/pgm/pgm_index.hpp:18:
./Index/PGM/pgm/piecewise_linear_model.hpp:31:9: warning: Compilation with -fopenmp is optional but recommended [-W#pragma-messages]
#pragma message ("Compilation with -fopenmp is optional but recommended")
        ^
./Index/PGM/index_test.cpp:123:37: warning: expression result unused [-Wunused-value]
            (multiset.contains(i) ? "true" : "false");
                                    ^~~~~~
2 warnings generated.


0

In [91]:
runtime_cost = np.loadtxt("runtime_result.txt")
f = open("runtime_result.txt",encoding="utf-8")
print(float(f.read()))
f.close
np.shape(runtime_cost)

0.003895


()

In [92]:
file = "./Index/PGM/index_test.cpp"
file_data = ""
with open(file, "r") as f:
        for line in f:
            line = line.replace("pgm::PGMIndex<K,%d, %d, float> pgm;"%(4478,75),"pgm::PGMIndex<K,%d, %d, float> pgm;" %(50,2) )
            file_data += line
with open(file,"w") as f:
        f.write(file_data)

#include <vector>

#include <cstdlib>

#include <iostream>

#include <algorithm>

#include <time.h>

#include <stdio.h>

#include <cstring>

#include <stdlib.h>

#include <fstream>

#include <cstdlib>

#include <sstream>





using namespace std;





#include "pgm/pgm_index.hpp"

#include "pgm/pgm_index_dynamic.hpp"

#include "pgm/pgm_index_variants.hpp"



template<typename K>

class PGMMultiset {

    std::vector<K> data;

    pgm::PGMIndex<K,50, 2, float> pgm;



public:



    explicit PGMMultiset(const std::vector<K> &data) : data(data), pgm(data.begin(), data.end()) {}



    bool contains(const K x) const {

        auto range = pgm.search(x);

        return std::binary_search(data.begin() + range.lo, data.begin() + range.hi, x);

    }



    auto lower_bound(const K x) const {

        auto range = pgm.search(x);

        return std::lower_bound(data.begin() + range.lo, data.begin() + range.hi, x);

    }



    auto upper_bound(const K x) const {

        auto range = pgm.s

In [94]:
a = "123.txt"
print(a+".txt")

123.txt.txt


In [102]:
a = np.load("./results/grid_search/result_data_0.npy")
a

array([3.03300000e-03, 6.48708812e+01])

In [128]:
a= list(np.arange(5,100,5))
b= list(np.arange(1,8000,100))

pd = {"a":a,"b":b}

    

result = itertools.product(pd["a"],pd["b"])

list_a = [k for k in result]

len(list_a)

print(list_a[1])





(5, 101)


In [129]:
from gym import spaces
action_space = spaces.Discrete(5)
action_space.n

5